In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
from pprint import pprint
import time
import re

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
scrape_dict = {}

### Nasa Mars News Scrape

In [5]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
time.sleep(1)
html = browser.html

In [6]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [7]:
#Grab the Title and teaser paragraph for first article on the website
title_results = soup.find_all('li', class_='slide')

#Create list variables to store
titles = []
news_p = []

#Loop through the title_results tags to identify the title and teaser body and append to respective lists above
for items in title_results:
    news_title = items.find('div', class_= 'content_title').a.text
    titles.append(news_title)
    
    news_teaser = items.find('div', class_='article_teaser_body').text
    news_p.append(news_teaser)

#Print first item in each list to verify
# print(titles[0])
# print(news_p[0])

#Set news_title and news_teaser variables with the first item in each list respectively
scrape_dict['news_title'] = titles[0]
scrape_dict['news_teaser'] = news_p[0]


### JPL Mars Space - Featured Image Scrape

In [8]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
time.sleep(1)
html = browser.html

In [9]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [10]:
#Find the article tag with class carousel item
img_results = soup.find_all('article', class_='carousel_item')

#Within the article tag pull out the text that is in the "style" tag
img = img_results[0]['style']

#Separate the string to only the url image
img_string = re.findall("'.*'", img_results[0]['style'])[0].replace("'","")

#Set variable with main url
main_url = 'https://www.jpl.nasa.gov'

#Concatenate the full url for the image
scrape_dict['featured_image_url'] = main_url + img_string

#Print out string to verify
# print(featured_image_url)


### Mars Weather Scrape

In [11]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(1)
html = browser.html

In [12]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [13]:
#Identify the html code that contains the tweets
tweet_results = soup.find_all('div', class_='js-tweet-text-container')

tweets = []

#Loop through all the tweets and pull out the tweet paragraph and append to tweets list
for tweet in tweet_results:
    new_tweets = tweet.find('p').text
    tweets.append(new_tweets)

#Print first item in tweets list to verify it's pulling the first tweet    
# print(tweets[0])

#Set mars_weather variable with first item in tweets list
scrape_dict['mars_weather'] = tweets[0]


### Mars Facts Scrape

In [14]:
url = 'https://space-facts.com/mars/'

In [15]:
#Read the url with pandas to find the table
tables = pd.read_html(url)
print(tables)

[                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.52 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                  -153 to 20 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers]


In [21]:
#Create a dataframe with the first table
table_df = tables[0]

#Rename the dataframe columns
table_df.columns = ['description', 'fact']

#Verify dataframe
# table_html = table_df.to_html()
# print(table_html)

#Convert the dataframe back to html string 
scrape_dict['table'] = table_df.to_html()


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>description</th>
      <th>fact</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
    

### Mars Hemispheres Scrape

In [17]:
#Set list and dictionaries to store scraped data
hemisphere_image_urls = []
image_dict1 = {}
image_dict2 = {}
image_dict3 = {}
image_dict4 = {}

In [18]:
#Go to desired url and click on the required page link to get the first image
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
time.sleep(2)
browser.click_link_by_partial_text('Cerberus Hemisphere Enhanced')
html = browser.html

In [19]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [20]:
#Identify the html tag for the needed information
container = soup.find('div', class_='container')

#Find the title of the desired image and append to a dictionary
name = container.find('h2', class_='title').text
image_dict1['title'] = name

#Find the image url and append to the dictionary
image = container.find('div', class_='downloads').li.a['href']
image_dict1['img_url'] = image

#Append the dictionary to the list
hemisphere_image_urls.append(image_dict1)

#Go back one page in the browser and then click into the next page
browser.click_link_by_partial_text('Back')
time.sleep(1)
browser.click_link_by_partial_text('Schiaparelli Hemisphere Enhanced')

#Repeat steps above three more times to get all the titles and images
html = browser.html
soup = bs(html, 'html.parser')

container = soup.find('div', class_='container')

name = container.find('h2', class_='title').text
image_dict2['title'] = name

image = container.find('div', class_='downloads').li.a['href']
image_dict2['img_url'] = image

hemisphere_image_urls.append(image_dict2)


browser.click_link_by_partial_text('Back')
time.sleep(1)
browser.click_link_by_partial_text('Syrtis Major Hemisphere Enhanced')

html = browser.html
soup = bs(html, 'html.parser')

container = soup.find('div', class_='container')

name = container.find('h2', class_='title').text
image_dict3['title'] = name

image = container.find('div', class_='downloads').li.a['href']
image_dict3['img_url'] = image

hemisphere_image_urls.append(image_dict3)

browser.click_link_by_partial_text('Back')
time.sleep(1)
browser.click_link_by_partial_text('Valles Marineris Hemisphere Enhanced')

html = browser.html
soup = bs(html, 'html.parser')

container = soup.find('div', class_='container')

name = container.find('h2', class_='title').text
image_dict4['title'] = name

image = container.find('div', class_='downloads').li.a['href']
image_dict4['img_url'] = image

hemisphere_image_urls.append(image_dict4)

scrape_dict['hemisphere_image_urls'] = hemisphere_image_urls

#Print hemispere_image_urls to verify and print scrape_dict to make sure everything is in the master dictionary
# print(hemisphere_image_urls)
print(scrape_dict)

{'news_title': 'Beyond Mars, the Mini MarCO Spacecraft Fall Silent', 'news_teaser': 'The first CubeSat mission to deep space may have reached their limit, but they could inspire future NASA missions.', 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16613-1920x1200.jpg', 'mars_weather': 'Happy National Weatherpersons Day to all my meteorologist friends.  Stay classy Gale Crater. https://en.wikipedia.org/wiki/National_Weatherperson%27s_Day\xa0…pic.twitter.com/IifCvlawci', 'table':             description                           fact
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.52 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                  -153 to 20 °C
7         First Record:              2nd mi